1. * This kernel is totally based on [feature-engineering-v-1-0](https://www.kaggle.com/ragnar123/feature-engineering-v-1-0)
and [xgboost-feature-selection-dsbowl](https://www.kaggle.com/shahules/xgboost-feature-selection-dsbowl) 
I used the feautres of first kernel and ensemble the result of Xgboost model and LGbm model.



In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold, StratifiedKFold
import gc
import json
pd.set_option('display.max_columns', 1000)

/kaggle/input/data-science-bowl-2019/sample_submission.csv
/kaggle/input/data-science-bowl-2019/specs.csv
/kaggle/input/data-science-bowl-2019/train_labels.csv
/kaggle/input/data-science-bowl-2019/train.csv
/kaggle/input/data-science-bowl-2019/test.csv


In [2]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
import shap

# Objective

* In the last notebook we create our baseline model including a feature selection part. 
* Cohen cappa score of 0.456 (lb) with a local cv score of 0.529
* In this notebook we are going to add more features and remove others that i think they overfitt the train set and then check if our local cv score improve.
* Next, we will check if this improvement aligns with the lb.

# Notes
* Check the distribution of the target variable of the out of folds score and the prediction distribution. A good model should more or less have the same distribution.

In [3]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))

    print('Reading train_labels.csv file....')
    train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
    print('Train_labels.csv file have {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))

    print('Reading specs.csv file....')
    specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
    print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, train_labels, specs, sample_submission

In [4]:
def encode_title(train, test, train_labels):
    # encode title
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    print(activities_map)
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    print(activities_labels)
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    print()
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    # convert text into datetime
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels

In [5]:
# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    # new features: time spent in each activity
    event_code_count = {eve: 0 for eve in list_of_event_code}
    last_session_time_sec = 0
    
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
                    
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(event_code_count.copy())
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        # this piece counts how many actions was made in each event_code so far
        n_of_event_codes = Counter(session['event_code'])
        
        for key in n_of_event_codes.keys():
            event_code_count[key] += n_of_event_codes[key]

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

In [6]:
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort = False)), total = 17000):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort = False), total = 1000):
        test_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    categoricals = ['session_title']
    return reduce_train, reduce_test, categoricals

In [7]:
def run_feature_selection(reduce_train, reduce_test, usefull_features, new_features):
    kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
    target = 'accuracy_group'
    oof_pred = np.zeros((len(reduce_train), 4))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = reduce_train[usefull_features].iloc[tr_ind], reduce_train[usefull_features].iloc[val_ind]
        y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature = categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature = categoricals)

        params = {
            'learning_rate': 0.01,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.75,
            'subsample': 0.75,
            'n_jobs': -1,
            'seed': 50,
            'max_depth': 10
        }

        model = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                          valid_sets=[train_set, val_set], verbose_eval = 500)
        oof_pred[val_ind] = model.predict(x_val)
    # using cohen_kappa because it's the evaluation metric of the competition
    loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
    score = loss_score
    usefull_new_features = []
    for i in new_features:
        oof_pred = np.zeros((len(reduce_train), 4))
        evaluating_features = usefull_features + usefull_new_features + [i]
        print('Evaluating {} column'.format(i))
        print('Out best cohen kappa score is : {}'.format(score))
        for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
            print('Fold {}'.format(fold + 1))
            x_train, x_val = reduce_train[evaluating_features].iloc[tr_ind], reduce_train[evaluating_features].iloc[val_ind]
            y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
            train_set = lgb.Dataset(x_train, y_train, categorical_feature = categoricals)
            val_set = lgb.Dataset(x_val, y_val, categorical_feature = categoricals)

            model = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                              valid_sets=[train_set, val_set], verbose_eval = 500)
            oof_pred[val_ind] = model.predict(x_val)
        loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
        print('Our new cohen kappa score is : {}'.format(loss_score))
        if loss_score > score:
            print('Feature {} is usefull, adding feature to usefull_new_features_list'.format(i))
            usefull_new_features.append(i)
            score = loss_score
        else:
            print('Feature {} is useless'.format(i))
        gc.collect()
    print('The best features are: ', usefull_new_features)
    print('Our best cohen kappa score is : ', score)

    return usefull_features + usefull_new_features

In [8]:
def run_lgb(reduce_train, reduce_test, usefull_features):
    kf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42)
    target = 'accuracy_group'
    oof_pred = np.zeros((len(reduce_train), 4))
    y_pred = np.zeros((len(reduce_test), 4))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = reduce_train[usefull_features].iloc[tr_ind], reduce_train[usefull_features].iloc[val_ind]
        y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=categoricals)

        params = {
            'learning_rate': 0.01,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.75,
            'subsample': 0.75,
            'n_jobs': -1,
            'seed': 50,
            'max_depth': 10
        }

        model = lgb.train(params, train_set, num_boost_round = 1000000, early_stopping_rounds = 50, 
                          valid_sets=[train_set, val_set], verbose_eval = 100)
        oof_pred[val_ind] = model.predict(x_val)
        y_pred += model.predict(reduce_test[usefull_features]) / 5
    loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
    result = pd.Series(np.argmax(oof_pred, axis = 1))
    print('Our oof cohen kappa score is: ', loss_score)
    print(result.value_counts(normalize = True))
    return y_pred



In [9]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
import shap

In [10]:
def run_xgb(reduce_train, final_test, usefull_features):
    n_splits=10
    scores=[]
    pars = {
        'colsample_bytree': 0.8,                 
        'learning_rate': 0.01,
        'max_depth': 10,
        'subsample': 1,
        'objective':'multi:softprob',
        'num_class':4,
        'eval_metric':'mlogloss',
        'min_child_weight':3,
        'gamma':0.25,
        'n_estimators':500
    }
    target = 'accuracy_group'
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    y_pre=np.zeros((len(final_test),4),dtype=float)
    final_test=xgb.DMatrix(final_test[usefull_features])
    
    for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
        print('Fold {}'.format(fold + 1))
        train_X, val_X = reduce_train[usefull_features].iloc[tr_ind], reduce_train[usefull_features].iloc[val_ind]
        train_y, val_y = reduce_train[target][tr_ind], reduce_train[target][val_ind]
        xgb_train = xgb.DMatrix(train_X, train_y)
        xgb_eval = xgb.DMatrix(val_X, val_y)

        xgb_model = xgb.train(pars,
                      xgb_train,
                      num_boost_round=1000,
                      evals=[(xgb_train, 'train'), (xgb_eval, 'val')],
                      verbose_eval=100,
                      early_stopping_rounds=20
                     )

        val_X=xgb.DMatrix(val_X)
        pred_val=[np.argmax(x) for x in xgb_model.predict(val_X)]
        score=cohen_kappa_score(pred_val,val_y,weights='quadratic')
        scores.append(score)
        print('choen_kappa_score :',score)

        pred=xgb_model.predict(final_test)
        y_pre+=pred

    pred = np.asarray([np.argmax(line) for line in y_pre])
    print('Mean score:',np.mean(scores))
    
    return xgb_model,pred


In [11]:
# ensemble of models


In [12]:
# read data
train, test, train_labels, specs, sample_submission = read_data()
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels = encode_title(train, test, train_labels)
# tranform function to get the train and test set

Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns
{'Watering Hole (Activity)': 0, 'Magma Peak - Level 2': 1, 'Mushroom Sorter (Assessment)': 2, 'Magma Peak - Level 1': 3, 'Cauldron Filler (Assessment)': 4, 'Chow Time': 5, 'All Star Sorting': 6, "Pirate's Tale": 7, 'Crystals Rule': 8, 'Crystal Caves - Level 3': 9, 'Bug Measurer (Activity)': 10, '12 Monkeys': 11, 'Heavy, Heavier, Heaviest': 12, 'Costume Box': 13, 'Chicken Balancer (Activity)': 14, 'Tree Top City - Level 3': 15, 'Scrub-A-Dub': 16, 'Tree Top City - Level 1': 17, 'Lifting Heavy Things': 18, 'Fireworks (Activity)': 19, 'Bird Measurer (Assessment)': 20, 'Leaf Leader': 21, 'Dino D

In [13]:
reduce_train, reduce_test, categoricals = get_train_and_test(train, test)

In [14]:
print(win_code)

{0: 4100, 1: 4100, 2: 4100, 3: 4100, 4: 4100, 5: 4100, 6: 4100, 7: 4100, 8: 4100, 9: 4100, 10: 4100, 11: 4100, 12: 4100, 13: 4100, 14: 4100, 15: 4100, 16: 4100, 17: 4100, 18: 4100, 19: 4100, 20: 4110, 21: 4100, 22: 4100, 23: 4100, 24: 4100, 25: 4100, 26: 4100, 27: 4100, 28: 4100, 29: 4100, 30: 4100, 31: 4100, 32: 4100, 33: 4100, 34: 4100, 35: 4100, 36: 4100, 37: 4100, 38: 4100, 39: 4100, 40: 4100, 41: 4100, 42: 4100, 43: 4100}


In [15]:
# function that creates more features
def preprocess(reduce_train, reduce_test):
    for df in [reduce_train, reduce_test]:
        df['installation_session_count'] = df.groupby(['installation_id'])['Clip'].transform('count')
        df['installation_duration_mean'] = df.groupby(['installation_id'])['duration_mean'].transform('mean')
        #df['installation_duration_std'] = df.groupby(['installation_id'])['duration_mean'].transform('std')
        df['installation_title_nunique'] = df.groupby(['installation_id'])['session_title'].transform('nunique')
        
        df['sum_event_code_count'] = df[[2050, 4100, 4230, 5000, 4235, 2060, 4110, 5010, 2070, 2075, 2080, 2081, 2083, 3110, 4010, 3120, 3121, 4020, 4021, 
                                        4022, 4025, 4030, 4031, 3010, 4035, 4040, 3020, 3021, 4045, 2000, 4050, 2010, 2020, 4070, 2025, 2030, 4080, 2035, 
                                        2040, 4090, 4220, 4095]].sum(axis = 1)
        
        df['installation_event_code_count_mean'] = df.groupby(['installation_id'])['sum_event_code_count'].transform('mean')
        #df['installation_event_code_count_std'] = df.groupby(['installation_id'])['sum_event_code_count'].transform('std')
        
    features = reduce_train.loc[(reduce_train.sum(axis=1) != 0), (reduce_train.sum(axis=0) != 0)].columns # delete useless columns
    features = [x for x in features if x not in ['accuracy_group', 'installation_id']]
    return reduce_train, reduce_test, features
# call feature engineering function
reduce_train, reduce_test, features = preprocess(reduce_train, reduce_test)

In [16]:
reduce_test.head()
# # next codes are to select only the feature that increase our cohen kappe score
# new_features = ['installation_session_count', 'installation_duration_mean', 'installation_duration_std', 'installation_title_nunique', 'sum_event_code_count', 'installation_event_code_count_mean', 
#                 'installation_event_code_count_std']
# usefull_features = [col for col in features if col not in new_features]
# best_features = run_feature_selection(reduce_train, reduce_test, usefull_features, new_features)

,Clip,Activity,Assessment,Game,2050,4100,4230,5000,4235,2060,4110,5010,2070,2075,2080,2081,2083,3110,4010,3120,3121,4020,4021,4022,4025,4030,4031,3010,4035,4040,3020,3021,4045,2000,4050,2010,2020,4070,2025,2030,4080,2035,2040,4090,4220,4095,installation_id,session_title,accumulated_correct_attempts,accumulated_uncorrect_attempts,duration_mean,accumulated_accuracy,accuracy_group,0,1,2,3,accumulated_accuracy_group,accumulated_actions,installation_session_count,installation_duration_mean,installation_title_nunique,sum_event_code_count,installation_event_code_count_mean
0,14,7,1,3,0,1,0,0,0,1,0,0,1,0,0,0,0,108,3,33,9,120,12,0,10,148,0,111,33,0,33,9,0,25,0,1,11,190,1,7,0,0,0,0,0,0,00abaee7,4,1,0,30.0,1.000000,0,0,0,0,1,3.0,867,1,30.0,1,867,867
1,29,11,5,12,6,23,0,4,0,4,3,4,1,3,10,2,7,416,12,25,72,325,26,36,94,407,11,418,49,25,25,72,6,57,0,4,85,385,2,80,0,2,6,2,9,0,01242218,24,4,7,49.6,0.466667,0,1,1,2,1,1.6,2718,1,49.6,1,2718,2718
2,6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,15,0,20,4,40,0,24,0,0,0,0,0,8,0,0,0,14,0,0,0,0,0,0,0,0,017c5718,2,0,0,0.0,0.000000,0,0,0,0,0,0.0,149,1,0.0,1,149,149
3,10,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,21,1,2,3,31,0,29,9,61,0,21,0,0,2,3,0,13,0,0,3,28,2,3,0,0,0,1,0,0,01a44906,2,0,0,0.0,0.000000,0,0,0,0,0,0.0,233,1,0.0,1,233,233
4,17,1,0,6,0,0,0,0,0,0,0,0,0,0,1,0,1,73,6,27,27,146,0,3,0,231,0,75,62,16,27,27,4,24,0,0,28,134,9,25,0,0,0,0,0,5,01bc6cb6,24,0,0,0.0,0.000000,0,0,0,0,0,0.0,951,1,0.0,1,951,951


In [17]:
y_lgb_pred = run_lgb(reduce_train, reduce_test, features)
#predict(reduce_test, sample_submission, y_pred)

Fold 1


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[100]	training's multi_logloss: 1.04846	valid_1's multi_logloss: 1.07028
[200]	training's multi_logloss: 0.972091	valid_1's multi_logloss: 1.01388
[300]	training's multi_logloss: 0.924775	valid_1's multi_logloss: 0.987083
[400]	training's multi_logloss: 0.889295	valid_1's multi_logloss: 0.974131
[500]	training's multi_logloss: 0.860156	valid_1's multi_logloss: 0.965842
[600]	training's multi_logloss: 0.834429	valid_1's multi_logloss: 0.961121
[700]	training's multi_logloss: 0.811617	valid_1's multi_logloss: 0.958433
[800]	training's multi_logloss: 0.790969	valid_1's multi_logloss: 0.956676
[900]	training's multi_logloss: 0.771431	valid_1's multi_logloss: 0.955332
[1000]	training's multi_logloss: 0.752824	valid_1's multi_logloss: 0.954463
Early stopping, best iteration is:
[1038]	training's multi_logloss: 0.746095	valid_1's multi_logloss: 0.954191
Fold 2
Training until validation scores don't improve for 50 rounds
[100]	traini

In [18]:
y_xgb_pred = run_xgb(reduce_train, reduce_test, features)
#predict(reduce_test, sample_submission, y_pred)

Fold 1


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:1.37862	val-mlogloss:1.38034
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 20 rounds.
[100]	train-mlogloss:0.938741	val-mlogloss:1.08727
[200]	train-mlogloss:0.739501	val-mlogloss:0.99806
[300]	train-mlogloss:0.619722	val-mlogloss:0.96998
[400]	train-mlogloss:0.541574	val-mlogloss:0.960408
Stopping. Best iteration:
[472]	train-mlogloss:0.500079	val-mlogloss:0.95784

choen_kappa_score : 0.5870557048555143
Fold 2
[0]	train-mlogloss:1.37866	val-mlogloss:1.38066
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 20 rounds.
[100]	train-mlogloss:0.936508	val-mlogloss:1.10799
[200]	train-mlogloss:0.732399	val-mlogloss:1.03208
[300]	train-mlogloss:0.614259	val-mlogloss:1.0118
[400]	train-mlogloss:0.535127	val-mlogloss:1.00663
Stopping. Best iteration:
[456]	train-mlogloss:0.501995	val-mlogloss:1.006

In [19]:
print(y_xgb_pred[1])

[3 3 3 3 3 3 3 3 0 0 3 3 3 3 0 0 3 3 3 3 1 3 3 3 1 0 3 0 3 3 0 0 0 1 3 3 0
 0 3 0 3 3 0 3 3 3 3 3 3 3 0 0 3 3 3 0 0 3 3 0 3 3 3 3 0 3 3 3 3 0 3 3 1 3
 3 3 0 3 3 0 3 3 3 3 3 3 3 0 3 3 3 3 3 3 0 3 3 3 0 3 3 0 3 3 0 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 0 3 3 3 3 3 0 3 3 3 3 3 1 0 3 3 3 3 0 0 3 3 3 0 0 3 3 1 3
 3 1 0 3 3 0 0 1 3 0 3 3 3 0 3 3 0 3 3 1 3 3 3 3 3 3 3 3 3 3 0 0 3 1 3 0 3
 3 3 3 3 3 3 0 0 0 0 3 2 0 1 3 3 3 0 0 3 0 3 0 3 3 3 3 0 3 3 3 3 1 0 0 3 3
 3 0 0 3 3 0 3 3 0 3 3 3 3 3 0 3 3 0 3 0 3 3 3 3 3 3 0 0 3 3 3 0 3 3 3 3 3
 3 0 3 3 0 3 3 3 3 0 0 0 3 3 0 3 3 0 0 3 3 3 3 0 0 3 3 3 0 3 3 3 3 3 3 3 3
 3 3 3 3 0 3 0 0 3 0 3 3 3 3 3 3 3 0 3 3 3 3 0 3 3 3 3 3 3 3 3 1 3 3 0 0 0
 3 3 3 3 3 3 0 3 3 3 0 3 3 3 0 3 3 3 0 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3
 3 3 0 3 3 3 3 0 3 3 3 3 3 1 3 0 3 3 3 3 3 0 1 3 3 3 1 0 0 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 0 3 1 0 3 0 3 3 1 3 3 3 0 3 3 0 3 0 3 1 3 0 0 3 3 0 3 3
 0 1 0 3 3 0 0 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 1 3 0 3 3 3 3 3 3 3 1 3 3 0 3
 3 3 3 3 0 1 3 0 3 0 3 3 

In [20]:
sub=pd.DataFrame({'installation_id':sample_submission.installation_id,'accuracy_group':y_xgb_pred[1]})

In [21]:
sub.head()

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3


In [22]:
def predict(reduce_test, sample_submission, y_pred):
    sample_submission['accuracy_group'] = np.round(y_pred).astype('int')
    sample_submission.to_csv('submission.csv', index = False)
    print(sample_submission['accuracy_group'].value_counts(normalize = True))

In [23]:
ensembled_prediction = y_lgb_pred.argmax(axis = 1)
predict(reduce_test, sample_submission, ensembled_prediction)

3    0.709
0    0.234
1    0.055
2    0.002
Name: accuracy_group, dtype: float64
